# Visualizing NPIs and cases evolution

### Simple visualization of NPIs stringency evolution with daily new cases increase according to the country (GeoID).

In [1]:
import pandas as pd
import os
import numpy as np

#Ignore pandas warnings
import warnings
warnings.simplefilter("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.dates as mdates
from matplotlib.dates import (DateFormatter, MonthLocator)

import ipywidgets as widgets
from ipywidgets import interact

%matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.dpi']= 150

In [2]:
#!conda install -n base -c conda-forge widgetsnbextension
#!conda install -n py36 -c conda-forge ipywidgets
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix

## Get data up to today as example

In [3]:
def add_GEOID(dataset):
    # Add RegionID column that combines CountryName and RegionName for easier manipulation of data
    dataset['GeoID'] = dataset['CountryName'] + '__' + dataset['RegionName'].astype(str)
    return dataset

In [4]:
url='https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
latest_df = pd.read_csv(url,parse_dates=['Date'],
                            encoding="ISO-8859-1",
                            dtype={"RegionName": str,
                                   "RegionCode": str},
                            error_bad_lines=False)
latest_df["RegionName"] = latest_df["RegionName"].fillna("")

NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing',
               'H6_Facial Coverings']
ID_COLUMNS = ['CountryName',
              'RegionName',
              'Date']

df = latest_df[ID_COLUMNS+NPI_COLUMNS+['ConfirmedCases']]

#Add geoID for easier manipulation of data
df = add_GEOID(df)

# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in NPI_COLUMNS:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))
    
# Compute the 7 day moving average
df["ActualDailyNewCases7DMA"] = df.groupby(
        "GeoID")['NewCases'].rolling(7, center=False).mean().reset_index(0, drop=True).fillna(0)

#Get final npis and cases csv (should be the same output format as predictions of cases or npis)
cases = df[ID_COLUMNS+["ActualDailyNewCases7DMA"]]
ip = df[ID_COLUMNS+NPI_COLUMNS]

#Add GeoID for easier manipulation of data
cases = add_GEOID(cases)
ip = add_GEOID(ip)

## Plots

In [5]:
#Labels for plot of NPIs evolution
ip_labels = ['School closing', 'Workplace closing','Cancel public events','Restrictions on gatherings',
'Close public transport','Stay at home requirements','Restrictions on internal movement',
'International travel controls','Public information campaigns','Testing policy','Contact tracing','Facial Coverings']

In [6]:
def plot_npi_evolution(country):
    # Pull out all relevant data for country selected
    ip_gdf = ip[ip.GeoID == country]
    npi_array = np.array(ip_gdf[NPI_COLUMNS])
    
    fig = plt.figure(figsize=(20,15))
    sns.set(font_scale=1.4)
    
    #First plot of new confirmed cases
    plt.subplot(211)
    plt.fill_between(df.Date.unique(), cases[cases.GeoID==country].ActualDailyNewCases7DMA, color ='steelblue')
    
    #Set our date locator by month because of space constraint
    loc = MonthLocator()
    formatter = DateFormatter('%d/%m/%y')
    
    ax = plt.gca()
    ax.xaxis.set_major_locator(loc)
    ax.xaxis.set_major_formatter(formatter)
    ax.xaxis.set_tick_params(rotation=20, labelsize=14, color='darkblue')
    ax.tick_params(labelcolor='darkblue')
    plt.ylabel('Daily New Cases (7DMA)', size=20, color='darkblue') #, plt.xlabel('Date', color ='darkblue', size=20)  
    
    #Plot the NPIs evolution through heatmap
    plt.subplot(212) 
    ax = sns.heatmap(npi_array.T, vmin =npi_array.min(), vmax= npi_array.max(), linewidths=.1,
                 xticklabels = (),
                 yticklabels = ip_labels, label=ip_labels,
                 cmap=sns.color_palette("Reds"), annot=False,
                 annot_kws={'size':30},
                cbar_kws={'fraction':0.05,'aspect':35, 'label':'NPIs stringency',"orientation": "horizontal",'ticks':[0,1,2,3,4],
                         'values':[0,1,2,3,4]})
    ax.tick_params(labelcolor='darkblue')
    plt.tight_layout()
    plt.show()

    
    
w_countries = widgets.Dropdown(options=df.GeoID.unique(), value='Aruba__', description='GeoID:', disabled=False)

interact(plot_npi_evolution, country=w_countries)


interactive(children=(Dropdown(description='GeoID:', options=('Aruba__', 'Afghanistan__', 'Angola__', 'Albania…

<function __main__.plot_npi_evolution(country)>

## To plot from predicted NPIs

In [ ]:
"""
#Import data
future_ips = pd.read_csv(predicted_npis_filepath, parse_dates=['Date'],
                            encoding="ISO-8859-1",
                            dtype={"RegionName": str,
                                   "RegionCode": str},
                            error_bad_lines=False)
                            
cases = pd.read_csv(cases_filepath, parse_dates=['Date'],
                            encoding="ISO-8859-1",
                            dtype={"RegionName": str,
                                   "RegionCode": str},
                            error_bad_lines=False)
                            
#Add GeoID for easier manipulation of data
cases = add_GEOID(cases)
ip = add_GEOID(future_ips)

#Plot
#Reminder: change columns names to plot from in 'plot_npi_evolution' function
w_countries = widgets.Dropdown(options=df.GeoID.unique(), value='Aruba__', description='GeoID:', disabled=False)
interact(plot_npi_evolution, country=w_countries)

"""